# Imports and Load Data

In [ ]:
# Standard scientific Python imports
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns # for visualisation
import numpy as np

from numpy import mean
from numpy import std
from numpy import arange

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE

from statsmodels.stats.outliers_influence import variance_inflation_factor

# Default plotting parameters
font = {'size'   : 18}
plt.rc('font', **font)

In [ ]:
# Load training dataset
rep = '/Users/Cherry0904/Desktop/MSc/Practicals/SML Practical/' 
y_train = pd.read_csv(rep + 'y_train.csv', index_col = 0, squeeze=True)
X_train = pd.read_csv(rep + 'X_train.csv', index_col = 0, header=[0, 1, 2]) #sets up header to be tuple
X_test = pd.read_csv(rep + 'X_test.csv', index_col = 0, header=[0, 1, 2])

# Create version with them together
Xy = pd.concat([X_train, y_train], axis = 1)

# StandardScaler
scaler = StandardScaler() 
scaler.fit(X_train)
X_train_sd = scaler.transform(X_train)
X_test_sd = scaler.transform(X_test)

# Extract variable names (in the form of 'feature-statistics-number')
variable_names = X_train.keys().get_level_values(0).astype(str).values + ' - ' + X_train.keys().get_level_values(1).astype(str).values + ' - ' + X_train.keys().get_level_values(2).astype(str).values
print(variable_names[0:5,])

['chroma_cens - kurtosis - 01' 'chroma_cens - kurtosis - 02'
 'chroma_cens - kurtosis - 03' 'chroma_cens - kurtosis - 04'
 'chroma_cens - kurtosis - 05']


# Functions

The Export Function:

In [ ]:
# Function to format the predictions in a dataframe and export to a csv file, to be uploaded on kaggle
def export_to_csv(y_hat, filename):
    df = pd.DataFrame({'Genre': y_hat})
    df.index.name = 'Id'
    df.to_csv(filename)

Define the function that assesses performance for each classifier, using three repeats of 4-fold CV:

In [ ]:
def cv_clf(clf, X, y):
    cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=3, random_state=15)
    # evaluate model accuracy
    cv_results = cross_validate(clf, X, y, scoring='accuracy', cv=cv, n_jobs=-1, return_train_score=True)
    ts_scores = cv_results['test_score']
    print('Mean Validation Accuracy: %.3f (%.3f)' % (mean(ts_scores), std(ts_scores)))

Define the grid search function for tuning parameters:

In [ ]:
def grid_search(clf, X, y, parameter, values):
    # define grid
    grid = dict()
    grid[parameter] = values
    # define search   
    cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=3, random_state=15)
    search = GridSearchCV(clf, grid, scoring='accuracy', cv=cv, n_jobs=-1)
    # perform the search
    results = search.fit(X, y)
    # summarize
    print('Mean Validation Accuracy: %.3f' % results.best_score_)
    print('Config: %s' % results.best_params_)

# QDA


Summary:
1. For Feature Selection, both ANOVA and RFE methods improve test accuracy, with ANOVA doing slightly better on the held-out test set (0.55).
2. Projecting the ANOVA-selected data on onto LDA discrimant coordinates further improves test accuracy (from 0.55 to 0.564). If only projecting the original (scaled) data onto LDA coordinates, it gives test accuracy of 0.556. Notice that there is no more colinearity issue using LDA coordinates.
3. No significant improvements by using different scalers.
4. No need to do polynomial input transformation, as it will certainly lead to overfitting.

### Feature Selection

In [ ]:
# Train-test split on scaled data
X_tr, X_te, y_tr, y_te = train_test_split(X_train, y_train, test_size=0.20 , random_state=15)

# Standardise data
scaler = StandardScaler() 
scaler.fit(X_tr)
X_tr_sd = scaler.transform(X_tr)
X_te_sd = scaler.transform(X_te)

Grid search the regularisation parameter. Regularisation parameter = 0.01 (does it mean the inverse?) offers a significant lift in performance.

0. All features used

In [ ]:
qda = QuadraticDiscriminantAnalysis(reg_param=0.0)
print("No Regularisation:")
cv_clf(qda, X_tr_sd, y_tr)

print("With Regularisation:")
values=(1e-06, 1e-05, 0.0001, 0.001, 0.01, arange(0, 1, 0.2))
grid_search(qda, X_tr_sd, y_tr, 'reg_param', values)

No Regularisation:
Mean Validation Accuracy: 0.144 (0.007)
With Regularisation:


/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Mean Validation Accuracy: 0.482
Config: {'reg_param': 0.01}


In [ ]:
# train the shrinkage model on the full training set from train-test split 
qda = QuadraticDiscriminantAnalysis(reg_param=0.01)
qda.fit(X_tr_sd,y_tr)
qda.score(X_te_sd,y_te) 

/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.5091666666666667

1. ANOVA

In [ ]:
# ANOVA feature selection 
fs = SelectKBest(score_func=f_classif, k=300)
fs.fit(X_tr_sd, y_tr)
X_tr_sel = fs.transform(X_tr_sd)
X_te_sel = fs.transform(X_te_sd)

In [ ]:
qda = QuadraticDiscriminantAnalysis(reg_param=0.0)
print("No Regularisation:")
cv_clf(qda, X_tr_sel, y_tr)

print("With Regularisation:")
values=(1e-06, 1e-05, 0.0001, 0.001, 0.01, arange(0, 1, 0.2))
grid_search(qda, X_tr_sel, y_tr, 'reg_param', values)

No Regularisation:
Mean Validation Accuracy: 0.433 (0.010)
With Regularisation:
Mean Validation Accuracy: 0.528
Config: {'reg_param': 0.01}


In [ ]:
# train the shrinkage model on the full training set from train-test split 
qda = QuadraticDiscriminantAnalysis(reg_param=0.01)
qda.fit(X_tr_sel,y_tr)
qda.score(X_te_sel,y_te) 

0.55

2. RFE

In [ ]:
# RFE feature selection
# here uses LDA as the estimator, as QDA does not have the 'coef_' attribute
rfe = RFE(estimator=LinearDiscriminantAnalysis(solver='eigen', store_covariance=True), n_features_to_select=300) 
rfe.fit(X_tr_sd, y_tr)
X_tr_sel = rfe.transform(X_tr_sd)
X_te_sel = rfe.transform(X_te_sd)

In [ ]:
qda = QuadraticDiscriminantAnalysis(reg_param=0.0)
print("No Regularisation:")
cv_clf(qda, X_tr_sel, y_tr)

print("With Regularisation:")
values=(1e-06, 1e-05, 0.0001, 0.001, 0.01, arange(0, 1, 0.2))
grid_search(qda, X_tr_sel, y_tr, 'reg_param', values)

No Regularisation:
Mean Validation Accuracy: 0.454 (0.011)
With Regularisation:
Mean Validation Accuracy: 0.532
Config: {'reg_param': 0.01}


In [ ]:
# train the shrinkage model on the full training set from train-test split 
qda = QuadraticDiscriminantAnalysis(reg_param=0.01)
qda.fit(X_tr_sel,y_tr)
qda.score(X_te_sel,y_te) 

0.5425

3. Remove Collinear Variables (with high VIF)

In [ ]:
# remove all features wit VIF greater than a threshold (to keep around 300 data points)
vif_train = pd.DataFrame()
vif_train["VIF Factor"] = [variance_inflation_factor(X_tr_sd, i) for i in range(X_tr_sd.shape[1])]

In [ ]:
index = vif_train["VIF Factor"] < 20
print(sum(index))
X_tr_sel = X_tr_sd[:, index] 
X_te_sel = X_te_sd[:, index] 

301


In [ ]:
qda = QuadraticDiscriminantAnalysis(reg_param=0.0)
print("No Regularisation:")
cv_clf(qda, X_tr_sel, y_tr)

print("With Regularisation:")
values=(1e-06, 1e-05, 0.0001, 0.001, 0.01, arange(0, 1, 0.2))
grid_search(qda, X_tr_sel, y_tr, 'reg_param', values)

No Regularisation:
Mean Validation Accuracy: 0.343 (0.022)
With Regularisation:
Mean Validation Accuracy: 0.472
Config: {'reg_param': 0.01}


/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [ ]:
# train the shrinkage model on the full training set from train-test split 
qda = QuadraticDiscriminantAnalysis(reg_param=0.01)
qda.fit(X_tr_sel,y_tr)
qda.score(X_te_sel,y_te)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.4625

### Project data on PCA/LDA components

In [ ]:
# define new data with selected features by ANOVA
X_tr_sd2 = fs.transform(X_tr_sd)
X_te_sd2 = fs.transform(X_te_sd)

# Fit PCA projections on training data, obtain projections for both train and test
k = 10
PC = PCA(n_components = k)
PC.fit(X_tr_sd2) 
ZPC_tr = PC.transform(X_tr_sd2) 
ZPC_te = PC.transform(X_te_sd2) 

# Fit LDA projections on training data, obtain projections for both train and test
LDA = LinearDiscriminantAnalysis(n_components = 7)
LDA.fit(X_tr_sd2, y_tr)
ZLDA_tr = LDA.transform(X_tr_sd2)
ZLDA_te = LDA.transform(X_te_sd2)

In [ ]:
print("First", k, "PCs:")
values=(1e-05, 0.0001, 0.001, 0.01, arange(0, 1, 0.2))
grid_search(qda, ZPC_tr, y_tr, 'reg_param', values)

print("LDA Components:")
values=(1e-05, 0.0001, 0.001, 0.01, arange(0, 1, 0.2))
grid_search(qda, ZLDA_tr, y_tr, 'reg_param', values)

First 10 PCs:
Mean Validation Accuracy: 0.437
Config: {'reg_param': 0.001}
LDA Components:
Mean Validation Accuracy: 0.647
Config: {'reg_param': 0.01}


In [ ]:
# train the LDA-transformed model on the full training set from train-test split 
qda = QuadraticDiscriminantAnalysis(reg_param=0.01)
qda.fit(ZLDA_tr,y_tr)
qda.score(ZLDA_te,y_te) 

0.5641666666666667

### Experiment with Other Scalers

Tried MinMaxScaler and QuantileTransformer. Overall, no other scalars have significantly improved test accuracy.

In [ ]:
# Train-test split
X_tr, X_te, y_tr, y_te = train_test_split(X_train, y_train, test_size = 0.20 , random_state=15)

# Normalise data - suffer from outliers
scaler2 = MinMaxScaler()
scaler2.fit(X_tr)
X_tr_nr = scaler2.transform(X_tr)
X_te_nr = scaler2.transform(X_te)

# Quantile-transform to uniform - robust to outliers
scaler3 = QuantileTransformer(output_distribution='uniform')
scaler3.fit(X_tr)
X_tr_qu = scaler3.transform(X_tr)
X_te_qu = scaler3.transform(X_te)

# Quantile-transform to normal - robust to outliers
scaler4 = QuantileTransformer(output_distribution='normal')
scaler4.fit(X_tr)
X_tr_qn = scaler4.transform(X_tr)
X_te_qn = scaler4.transform(X_te)

ANOVA, LDA, Grid search and fit on test for Normalised data:

In [ ]:
# define new data with selected features by ANOVA
fs.fit(X_tr_nr, y_tr)
X_tr_nr2 = fs.transform(X_tr_nr)
X_te_nr2 = fs.transform(X_te_nr)

LDA.fit(X_tr_nr2, y_tr)
Z_tr = LDA.transform(X_tr_nr2)
Z_te = LDA.transform(X_te_nr2)

In [ ]:
# Grid search
qda = QuadraticDiscriminantAnalysis()
values=(1e-07, 1e-06, 1e-05, 0.0001, 0.001, 0.01, arange(0, 1, 0.2))
grid_search(qda, Z_tr, y_tr, 'reg_param', values)

Mean Validation Accuracy: 0.647
Config: {'reg_param': 0.01}


In [ ]:
# validation accuracy
qda = QuadraticDiscriminantAnalysis(reg_param=0.01)
qda.fit(Z_tr,y_tr)
qda.score(Z_te,y_te)

0.5641666666666667

ANOVA, LDA, Grid search and fit on test for Uniform-Scaled data:

In [ ]:
# For Uniform-Scaled data
# define new data with selected features by ANOVA
fs.fit(X_tr_qu, y_tr)
X_tr_qu2 = fs.transform(X_tr_qu)
X_te_qu2 = fs.transform(X_te_qu)

LDA.fit(X_tr_qu2, y_tr)
Z_tr = LDA.transform(X_tr_qu2)
Z_te = LDA.transform(X_te_qu2)

In [ ]:
# Grid search
qda = QuadraticDiscriminantAnalysis()
values=(1e-07, 1e-06, 1e-05, 0.0001, 0.001, 0.01, arange(0, 1, 0.2))
grid_search(qda, Z_tr, y_tr, 'reg_param', values)

Mean Validation Accuracy: 0.657
Config: {'reg_param': 0.01}


In [ ]:
# validation accuracy
qda = QuadraticDiscriminantAnalysis(reg_param=0.01)
qda.fit(Z_tr,y_tr)
qda.score(Z_te,y_te)

0.5508333333333333

ANOVA, LDA, Grid search and fit on test for Normal-Scaled data:

In [ ]:
# For Normal-Scaled data
# define new data with selected features by ANOVA
fs.fit(X_tr_qn, y_tr)
X_tr_qn2 = fs.transform(X_tr_qn)
X_te_qn2 = fs.transform(X_te_qn)

LDA.fit(X_tr_qn2, y_tr)
Z_tr = LDA.transform(X_tr_qn2)
Z_te = LDA.transform(X_te_qn2)

In [ ]:
# Grid search
qda = QuadraticDiscriminantAnalysis()
values=(1e-10, 1e-09, 1e-08, 1e-07, 1e-06, 1e-05, 0.0001, 0.001, 0.01, arange(0, 1, 0.2))
grid_search(qda, Z_tr, y_tr, 'reg_param', values)

Mean Validation Accuracy: 0.658
Config: {'reg_param': 1e-10}


In [ ]:
# validation accuracy
qda = QuadraticDiscriminantAnalysis(reg_param=1e-10)
qda.fit(Z_tr,y_tr)
qda.score(Z_te,y_te) # slightly better than StandardScalar but difference is too small

0.565

Fit the model on the full training data:

In [ ]:
# StandardScaler
scaler = StandardScaler() 
scaler.fit(X_train)
X_train_sd = scaler.transform(X_train)
X_test_sd = scaler.transform(X_test)

fs = SelectKBest(score_func=f_classif, k=300)
fs.fit(X_train_sd, y_train)
X_train_sd2 = fs.transform(X_train_sd)
X_test_sd2 = fs.transform(X_test_sd)

LDA = LinearDiscriminantAnalysis(n_components = 7)
LDA.fit(X_train_sd2, y_train)
Z_train = LDA.transform(X_train_sd2)
Z_test = LDA.transform(X_test_sd2)

QDA = QuadraticDiscriminantAnalysis(reg_param=0.01)
QDA.fit(Z_train, y_train)
y_hat = QDA.predict(Z_test)

# Export to CSV file 
export_to_csv(y_hat,'Predictions_QDA_final.csv')